In [1]:
import requests

import json

import pandas as pd

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

/var/folders/6g/94mzvld16m75jj2nk4wk6jnr0000gp/T/ipykernel_13602/3821916218.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [20]:
api_key = "cbcffe5648e04b07afa203548241202"  # API key obtained from https://www.worldweatheronline.com/
location_list = [
    "02459"
]  # list of strings containg US Zipcode, UK Postcode, Canada Postalcode, IP address, Latitude/Longitude (decimal degree) or city name
start_date = "01-JAN-2021"  # date when desired scraping period starts; preferred date format: 'dd-MMM-yyyy', e.g., '01-JAN-2020'
end_date = "31-DEC-2022"  # date when desired scraping period ends; preferred date format: 'dd-MMM-yyyy
frequency = 24  # frequency between observations; possible values 1 (1 hour), 3 (3 hours), 6 (6 hours), 12 (12 hours (day/night)) or 24 (daily averages)

In [3]:
import datetime as dt
from dateutil.relativedelta import relativedelta

In [4]:
dt.datetime.strftime(dt.datetime.today(), "%d-%b-%Y").upper().title()

'13-Feb-2024'

In [6]:
import datetime as dt
from projects.weather.wwo_utils import split_date_range


dt_start = dt.datetime(1215, 6, 15)
dt_end = dt.datetime(1216, 1, 22)
split_date_range(dt_start, dt_end)
[
    "15-JUN-1215",
    "01-JUL-1215",
    "01-AUG-1215",
    "01-SEP-1215",
    "01-OCT-1215",
    "01-NOV-1215",
    "01-DEC-1215",
    "01-JAN-1216",
    "22-JAN-1216",
]

['15-JUN-1215',
 '01-JUL-1215',
 '01-AUG-1215',
 '01-SEP-1215',
 '01-OCT-1215',
 '01-NOV-1215',
 '01-DEC-1215',
 '01-JAN-1216',
 '22-JAN-1216']

In [21]:
payload = {
    "key": api_key,
    "date": start_date,
    "enddate": end_date,
    "q": location_list[0],
}

In [22]:
end_date

'31-DEC-2022'

In [23]:
response = requests.get(
    f"https://api.worldweatheronline.com/premium/v1/past-weather.ashx?key={api_key}&date={start_date}&enddate={end_date}&q={location_list[0]}&tp=1&format=json&includelocation=yes"
)

In [17]:
response.content

b'{"data":{"request":[{"type":"Zipcode","query":"02459"}],"nearest_area":[{"areaName":[{"value":"Newton Center"}],"country":[{"value":"USA"}],"region":[{"value":"Massachusetts"}],"latitude":"42.309","longitude":"-71.192"}],"weather":[{"date":"2020-01-01","astronomy":[{"sunrise":"07:14 AM","sunset":"04:22 PM","moonrise":"11:15 AM","moonset":"10:49 PM","moon_phase":"Waxing Crescent","moon_illumination":"37"}],"maxtempC":"3","maxtempF":"38","mintempC":"-1","mintempF":"31","avgtempC":"1","avgtempF":"34","totalSnow_cm":"0.0","sunHour":"9.0","uvIndex":"1","hourly":[{"time":"0","tempC":"2","tempF":"36","windspeedMiles":"10","windspeedKmph":"15","winddirDegree":"250","winddir16Point":"WSW","weatherCode":"119","weatherIconUrl":[{"value":"https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0003_white_cloud.png"}],"weatherDesc":[{"value":"Cloudy"}],"precipMM":"0.0","precipInches":"0.0","humidity":"82","visibility":"10","visibilityMiles":"6","pressure":"1000","pressureInches":"30",

In [11]:
daily_weather_vars=['date','maxtempC', 'maxtempF', 'mintempC', 'mintempF',
       'avgtempC', 'avgtempF', 'totalSnow_cm', 'sunHour', 'uvIndex', ]
astronomy_vars=['sunrise', 'sunset', 'moonrise', 'moonset', 'moon_phase', 'moon_illumination']
hourly_vars=['time', 'tempC', 'tempF', 'windspeedMiles', 'windspeedKmph', 'winddirDegree', 'winddir16Point', 'weatherCode', 'weatherDesc', 'precipMM', 'precipInches', 'humidity', 'visibility', 'visibilityMiles', 'pressure', 'pressureInches', 'cloudcover', 'HeatIndexC', 'HeatIndexF', 'DewPointC', 'DewPointF', 'WindChillC', 'WindChillF', 'WindGustMiles', 'WindGustKmph', 'FeelsLikeC', 'FeelsLikeF', 'uvIndex']
extract_weather_desc=False
if 'weatherDesc' in hourly_vars:
       hourly_vars=[x for x in hourly_vars if x != 'weatherDesc']
       extract_weather_desc=True

In [ ]:
mydict={'locations': ['02459'],
'api_key': "cbcffe5648e04b07afa203548241202"  ,
'entry_point': "https://api.worldweatheronline.com/premium/v1/past-weather.ashx"  ,
'start_date' : "01-JAN-2020" ,
'end_date' : "31-DEC-2022" ,
'timeout_seconds': 30,
'frequency' : 24 ,
'daily_weather_variables':['date','maxtempC', 'maxtempF', 'mintempC', 'mintempF',
       'avgtempC', 'avgtempF', 'totalSnow_cm', 'sunHour', 'uvIndex', ]}

In [ ]:
from schema import Schema, And, Use, Optional, SchemaError
import yaml
import validators
schema = Schema(
        {
            'locations': And(list, lambda x: all([isinstance(xx,str) for x in xx])),
            'api_key': And(str),
            Optional('entry_point',default='https://api.worldweatheronline.com/premium/v1/past-weather.ashx'): And(str,lambda x : validators.url(x)),
            'end_date': And(str,lambda x: len(x)==11),
            'start_date': And(str,lambda x: len(x)==11),
            Optional('timeout_seconds',default=30): And(int, lambda t: t>0),
            Optional('frequency',default=1): And(lambda f : f in [1, 3, 6, 12, 24],int),
            'daily_weather_variables': And(list, lambda x: all([xx in ['date','maxtempC', 'maxtempF', 'mintempC', 'mintempF','avgtempC', 'avgtempF', 'totalSnow_cm', 'sunHour', 'uvIndex'] for xx in x]) ),
            'astronomy_variables': And(list, lambda x: all([xx in ['sunrise', 'sunset', 'moonrise', 'moonset', 'moon_phase', 'moon_illumination'] for xx in x]) ),
            'hourly_variables': And(list, lambda x: all([xx in ['time', 'tempC', 'tempF', 'windspeedMiles', 'windspeedKmph', 'winddirDegree', 'winddir16Point', 'weatherCode', 'weatherDesc', 'precipMM', 'precipInches', 'humidity', 'visibility', 'visibilityMiles', 'pressure', 'pressureInches', 'cloudcover', 'HeatIndexC', 'HeatIndexF', 'DewPointC', 'DewPointF', 'WindChillC', 'WindChillF', 'WindGustMiles', 'WindGustKmph', 'FeelsLikeC', 'FeelsLikeF', 'uvIndex'] for xx in x]) )
        }
    )

In [24]:


j = response.json()

df = pd.DataFrame.from_dict(j)

In [25]:
df

,data
request,"[{'type': 'Zipcode', 'query': '02459'}]"
nearest_area,"[{'areaName': [{'value': 'Newton Center'}], 'c..."
weather,"[{'date': '2021-01-01', 'astronomy': [{'sunris..."


In [26]:


weather=json.loads(response.content.decode('utf-8'))['data']['weather']

weather_list=[]

for i in range(len(weather)):
    weather_dict={k : weather[i][k] for k in tuple(daily_weather_vars)}
    weather_dict.update({k : weather[i]['astronomy'][0][k] for k in tuple(astronomy_vars)})
    for h in range(len(weather[i]['hourly'])):
        weather_dict.update({k : weather[i]['hourly'][h][k] for k in tuple(hourly_vars)})
        if extract_weather_desc:
            weather_dict.update({'weatherDesc':weather[i]['hourly'][h]['weatherDesc'][0]['value']})
        weather_list.append(weather_dict.copy())
weather_df=pd.DataFrame(weather_list)



In [27]:
weather_df.date.value_counts()

date
2021-01-01    24
2021-01-27    24
2021-01-21    24
2021-01-22    24
2021-01-23    24
2021-01-24    24
2021-01-25    24
2021-01-26    24
2021-01-28    24
2021-01-19    24
2021-01-29    24
2021-01-30    24
2021-01-31    24
2021-02-01    24
2021-02-02    24
2021-02-03    24
2021-01-20    24
2021-01-18    24
2021-01-02    24
2021-01-09    24
2021-01-03    24
2021-01-04    24
2021-01-05    24
2021-01-06    24
2021-01-07    24
2021-01-08    24
2021-01-10    24
2021-01-17    24
2021-01-11    24
2021-01-12    24
2021-01-13    24
2021-01-14    24
2021-01-15    24
2021-01-16    24
2021-02-04    24
Name: count, dtype: int64

In [ ]:
with open("/Users/lindseygulden/dev/leg-up/data/newton_tp1.json", "wb") as fd:
    for chunk in response.iter_content(chunk_size=128):
        fd.write(chunk)

In [ ]:
# df = pd.read_json("/Users/lindseygulden/dev/leg-up/data/newton_tp24.json")
df = pd.read_json("/Users/lindseygulden/dev/leg-up/data/newton_tp1.json")
df

In [ ]:
h_list=[]
w_list=[]
n_entries=len(df.loc['weather',]['data'])
for d in range(n_entries):
    w_list.append(df.loc['weather',]['data'][d])

all_weather_df=pd.DataFrame(w_list)
forecast_vars=['date','maxtempF','mintempF','avgtempF','totalSnow_cm','sunHour','uvIndex']


In [ ]:
n_hours=all_weather_df.loc[0,'hourly']

In [11]:
import yaml
with open('/Users/lindseygulden/dev/leg-up/projects/weather/config/data_description.yml','r') as f:
    try:
        tmp=yaml.safe_load(f)    
    except yaml.YAMLError as e:
        print(e)


In [12]:
print(tmp)

{'DewPointC': 'dew_point_c', 'DewPointF': 'dew_point_f', 'FeelsLikeC': 'feels_like_c', 'FeelsLikeF': 'feels_like_f', 'HeatIndexC': 'heat_index_c', 'HeatIndexF': 'heat_index_f', 'WindChillC': 'wind_chill_c', 'WindChillF': 'wind_chill_f', 'WindGustKmph': 'wind_gust_kmph', 'WindGustMiles': 'wind_gust_mph', 'avgtempC': 'avg_temp_c', 'avgtempF': 'avg_temp_f', 'cloudcover': 'cloud_cover', 'date': 'date', 'humidity': 'humidity', 'maxtempC': 'max_temp_c', 'maxtempF': 'max_temp_f', 'mintempC': 'min_temp_c', 'mintempF': 'min_temp_f', 'moon_illumination': 'moon_illumination', 'moon_phase': 'moon_phase', 'moonrise': 'moonrise_local_time', 'moonset': 'moonset_local_time', 'precipInches': 'precip_in', 'precipMM': 'precip_mm', 'pressure': 'pressure_mb', 'pressureInches': 'pressure_in', 'sunHour': 'sun_hour', 'sunrise': 'sunrise_local_time', 'sunset': 'sunset_local_time', 'tempC': 'temp_c', 'tempF': 'temp_f', 'time': 'time', 'totalSnow_cm': 'total_snow_cm', 'uvIndex': 'uv_index', 'visibility': 'visibi

In [ ]:

with open('../../config.yml', 'w') as outfile:
    yaml.dump(hourly[0], outfile, default_flow_style=False)
    


In [ ]:
daily_weather_vars=['date','maxtempC', 'maxtempF', 'mintempC', 'mintempF',
       'avgtempC', 'avgtempF', 'totalSnow_cm', 'sunHour', 'uvIndex', ]
astronomy_vars={'sunrise':'sunrise_local_time', 'sunset':'sunset_local_time', 'moonrise':'moonrise_local_time', 
                'moonset':'moonset_local_time', 'moon_phase':'moon_phase', 'moon_illumination':'moon_illumination'}
hourly_vars=['time', 'tempC', 'tempF', 'windspeedMiles', 'windspeedKmph', 'winddirDegree', 'winddir16Point', 'weatherCode', 'weatherDesc', 'precipMM', 'precipInches', 'humidity', 'visibility', 'visibilityMiles', 'pressure', 'pressureInches', 'cloudcover', 'HeatIndexC', 'HeatIndexF', 'DewPointC', 'DewPointF', 'WindChillC', 'WindChillF', 'WindGustMiles', 'WindGustKmph', 'FeelsLikeC', 'FeelsLikeF', 'uvIndex']


In [ ]:
all_weather_df.loc[0,'astronomy']

In [ ]:

h_list=[]
w_list=[]
for d in range(35):
    w_list.append(df.loc['weather',]['data'][d])

all_weather_df=pd.DataFrame(w_list)
forecast_vars=['date','maxtempC', 'mintempC', 'avgtempC','maxtempF','mintempF','avgtempF','totalSnow_cm','sunHour','uvIndex']
forecast_df=all_weather_df[forecast_vars]


for d in range(35):
    hourly=all_weather_df.loc[d,'hourly']
    astronomy=all_weather_df.loc[d,'astronomy'][0]
    today_date=all_weather_df.loc[d,'date']
    for hour in hourly:
        # add daily variables to hourly variables
        for v in astronomy_vars:
            hour[v]=astronomy[v]
        for v in forecast_vars:
            hour[v]=all_weather_df.loc[d,v]
        h_list.append(hour)
hourly_df=pd.DataFrame(h_list)
hourly_df=hourly_df[daily_weather_vars+hourly_vars]

In [ ]:
hourly_df

In [ ]:
config = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up/config/weather/newton_weather.yml"
)

In [ ]:
hourly_df.date

In [ ]:
all_weather_df.loc[0, forecast_vars]

In [ ]:
all_weather_df.loc[0, "hourly"]

In [ ]:
hourly_df.loc

In [ ]:
for i in hourly_df.columns.values:
    print(f"   - '{i}'")

In [ ]:
import sys

print(sys.path)

import sys
print(sys.path)

In [ ]:
from projects.utils.io import yaml_to_dict

In [ ]:
tmp = pd.DataFrame(w_list)
len(tmp.loc[0, "hourly"])